In [ ]:
import pandas as pd
from datetime import datetime
import os
import time
start_time = time.time()
# Set working directory to where the python file is located
wd = os.path.dirname(os.path.abspath(__file__))
os.chdir(wd)
input_path_main = 'output_2.xlsx'
df = pd.read_excel(input_path_main)
42
df.rename(columns={'date': 'checkin_dates'}, inplace=True)
print(df['checkin_dates'])
# Define timeframe (same as in first scripts)
timeframe_start = '2018-01-01'
timeframe_end = '2019-12-31'
# Create dates_during_timeframe column on which the df can be exploded later
def filter_dates(date_list):
if isinstance(date_list, str):
filtered_dates = []
for date_str in date_list.split(', '):
try:
date = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S').date()
if datetime.strptime(timeframe_start, '%Y-%m-%d').date() <= date <= datetime.strptime(timeframe_end, '%Y-%m-%d').date():
filtered_dates.append(date.strftime('%Y-%m-%d'))
except ValueError as e:
print(f"Error parsing date: {date_str} - {e}")
return filtered_dates
else:
return []
df['dates_during_timeframe'] = df['checkin_dates'].apply(filter_dates)
# Get the checkins count for each restaurant during the analyzed timeframe
df['checkins_during_timeframe'] = df['dates_during_timeframe'].apply(len)
# export data
with pd.ExcelWriter('output_3.xlsx') as writer:
df.to_excel(writer, sheet_name='sheet_1', index=False)
df.to_csv('output_3.csv', index=False)
print('--- Runtime: %s seconds ---' % (time.time() - start_time))
t-test
import pandas as pd
43
from scipy.stats import ttest_ind
import os
import time
start_time = time.time()
wd = os.path.dirname(os.path.abspath(__file__))
os.chdir(wd)
input_path_main = 'result_df.csv'
df = pd.read_csv(input_path_main)
# Recoding weather_code
df['weather_good_bad'] = df.apply(lambda row: 1 if (row['weather_code'] < 20 and (16 <= row['temperature_2m_mean'] <= 26)) else 0, axis=1)
print(df)
df_h2 = df[df['weather_good_bad'] == 1]
print(df_h2)
print(list(df_h2))
df_h2_outdoor_seating = df_h2[df_h2['attributes.OutdoorSeating'] == 1]
df_h2_no_outdoor_seating = df_h2[df_h2['attributes.OutdoorSeating'] == 0]
ttest_h2 = ttest_ind(df_h2_outdoor_seating['daily_checkin_count'], df_h2_no_out-door_seating['daily_checkin_count'])
print(f"t-Test H2: {ttest_h2}")
print(df_h2_outdoor_seating['daily_checkin_count'].mean())
print(df_h2_no_outdoor_seating['daily_checkin_count'].mean())
44
# Visualization (Boxplots)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
df_h2_outdoor_seating_vis = df_h2_outdoor_seating[['attributes.OutdoorSeating']]
df_h2_no_outdoor_seating_vis = df_h2_no_outdoor_seating[['attributes.OutdoorSeat-ing']]
# Create a df for visualization
df_h2_vis = pd.DataFrame({
'Daily Checkin Count': np.concatenate([df_h2_outdoor_seat-ing['daily_checkin_count'],
df_h2_no_outdoor_seating['daily_checkin_count']]),
'Group': ['Outdoor Seating'] * len(df_h2_outdoor_seating) +
['No Outdoor Seating'] * len(df_h2_no_outdoor_seating)
})
# Creating a box plot
plt.figure(figsize=(10, 6))
sns.boxplot(x='Group', y='Daily Checkin Count', data=df_h2_vis)
plt.title('Comparison of Daily Checkin Counts (Hypothesis H2)')
plt.show()
data_outdoor_seating = df_h2_outdoor_seating['daily_checkin_count']
data_no_outdoor_seating = df_h2_no_outdoor_seating['daily_checkin_count']
# Set the number of bins for the histogram
bins = 20
fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
# Histogram for outdoor seating
axs[0].hist(data_outdoor_seating, bins=bins, color='skyblue', edgecolor='black')
axs[0].set_title('Outdoor Seating')
axs[0].set_xlabel('Daily Checkin Count')
axs[0].set_ylabel('Frequency')
45
# Histogram for no outdoor seating
axs[1].hist(data_no_outdoor_seating, bins=bins, color='salmon', edgecolor='black')
axs[1].set_title('No Outdoor Seating')
axs[1].set_xlabel('Daily Checkin Count')
plt.suptitle('Histograms of Daily Checkin Counts')
plt.show()
df['weather_rain'] = df['weather_code'].apply(lambda x: 1 if x >= 20 else 0)
df_h3 = df[df['weather_rain'] == 1]
print(df_h3)
df_h3_parking = df_h3[df_h3['attributes.BusinessParking.Garage'] == 0]
df_h3_no_parking = df_h3[df_h3['attributes.BusinessParking.Garage'] == 1]
ttest_h3 = ttest_ind(df_h3_parking['daily_checkin_count'], df_h3_no_park-ing['daily_checkin_count'])
print(f"t-Test H3: {ttest_h3}")
print(df_h3_parking['daily_checkin_count'].mean())
print(df_h3_no_parking['daily_checkin_count'].mean())
print('--- Runtime: %s seconds ---' % (time.time() - start_time))